## Alcholic Anonymous News articles extraction from BBC. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions.

### Lists for Alcholic Anonymous News Articles extracted data

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results

In [3]:
keyword = 'Alcoholics Anonymous site:www.bbc.co.uk'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
max_pages = round([int(s) for s in soup.select_one('#resultStats').text.split() if s.isdigit()][0]/10)

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### Finding the total no.of.pages by total no.of articles from google search results

In [5]:
keyword = 'Alcoholics Anonymous site:www.bbc.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
max_pages = round([int(s) for s in soup.select_one('#resultStats').text.split() if s.isdigit()][0]/10)

### Iterates max_pages value through while loop. Scraping the Articles urls

In [6]:
index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [7]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

401 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [8]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(article.title.strip())
            except:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('meta[name="author"]')['content'].strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.storyText').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\jieba\dict.txt ...
Dumping model to file cache C:\Users\GM\AppData\Local\Temp\jieba.cache
Loading model cost 4.021788120269775 seconds.
Prefix dict has been built succesfully.


Wall time: 33min 41sc.com/news/live/uk-england-beds-bucks-herts-3289775725.stm-tell-if-youre-a-workaholic


### Checking Array Length of each list to create DataFrame

In [10]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

401 401 401 401 401 401 401 401


### Creating a csv file after checking array length and droping the missing values from the dataset

In [11]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,
                    'Source_URLs' : urls})

tbl.dropna()
tbl.to_csv('BBC.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles,Keywords,Summaries,Source_URLs
0,Every university 'needs' Alcoholics Anonymous ...,Alcoholics Anonymous says the number of younge...,None,2018-07-31T16:54:32+00:00,Image copyright Getty Images\n\nAlcoholics Ano...,"[aa, meetings, student, wasnt, really, preside...",Image copyright Getty ImagesAlcoholics Anonymo...,https://www.bbc.co.uk/news/amp/uk-wales-45023075
1,What happens in an Alcoholics Anonymous meeting?,Members of Alcoholics Anonymous are marking it...,None,None,Image caption The true nature of alcoholism is...,"[aa, drink, woman, bad, life, god, meeting, dr...",Image caption The true nature of alcoholism is...,https://www.bbc.co.uk/news/10280806
2,The many groups that have copied Alcoholics An...,Alcoholics Anonymous was founded 80 years ago....,None,None,Image copyright Other Image caption An AA meet...,"[aa, meeting, 12, addiction, groups, 12step, a...","But Peter hasn't tasted alcohol in 34 years, a...",https://www.bbc.co.uk/news/magazine-33049093
3,Alcoholics Anonymous,,None,None,,"[alcoholics, anonymous]",,https://www.bbc.co.uk/programmes/topics/Alcoho...
4,Sir Anthony Hopkins opens up on alcohol battle,"The Oscar-winning star says he was ""disgusted,...",None,2018-07-27T09:28:17+00:00,Image copyright Greg Doherty / Getty Images Im...,"[image, sir, alcohol, early, woman, battle, ho...",Image copyright Greg Doherty / Getty Images Im...,https://www.bbc.co.uk/news/amp/uk-wales-south-...
